In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use('seaborn-whitegrid')

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
from scipy.integrate import odeint

def lorenz(x, t, F):
    '''Partial derivatives for Lorenz-96 ODE.'''
    p = len(x)
    dxdt = np.zeros(p)
    for i in range(p):
        dxdt[i] = (x[(i+1) % p] - x[(i-2) % p]) * x[(i-1) % p] - x[i] + F

    return dxdt

def simulate_lorenz_96(p, T, F=10.0, delta_t=0.1, sd=0.1, burn_in=1000,
                       seed=0):
    if seed is not None:
        np.random.seed(seed)

    # Use scipy to solve ODE.
    x0 = np.random.normal(scale=0.01, size=p)
    t = np.linspace(0, (T + burn_in) * delta_t, T + burn_in)
    X = odeint(lorenz, x0, t, args=(F,))
    X += np.random.normal(scale=sd, size=(T + burn_in, p))

    # Set up Granger causality ground truth.
    GC = np.zeros((p, p), dtype=int)
    for i in range(p):
        GC[i, i] = 1
        GC[i, (i + 1) % p] = 1
        GC[i, (i - 1) % p] = 1
        GC[i, (i - 2) % p] = 1

    return X[burn_in:], GC

In [3]:
X_np, GC = simulate_lorenz_96(p=10, F=10, T=1000)
X = torch.tensor(X_np[np.newaxis], dtype=torch.float64, device=device)
X = X.reshape(10,1000)
input_ = torch.arange(0,1000, 1, device = device, dtype = float)

In [4]:
X.size()

torch.Size([10, 1000])

In [5]:
model_pred = torch.rand((10,1000), device = device, dtype = torch.float64)

In [6]:
def make_dataset(model_pred, True_target):
    Y = True_target
    X = True_target - model_pred

    return X, Y

In [7]:
input, R_X = make_dataset(model_pred, X)

In [9]:
def arrange_input(data, context):
    '''
    Arrange a single time series into overlapping short sequences.
    Args:
      data: time series of shape (T, dim).
      context: length of short sequences.
    '''
    assert context >= 1 and isinstance(context, int)
    input = torch.zeros(len(data) - context, context, data.shape[1],
                        dtype=torch.float32, device=data.device)
    target = torch.zeros(len(data) - context, context, data.shape[1],
                         dtype=torch.float32, device=data.device)
    for i in range(context):
        start = i
        end = len(data) - context + i
        input[:, i, :] = data[start:end]
        target[:, i, :] = data[start+1:end+1]
    return input.detach(), target.detach()

In [10]:
input_ori, target_ori = arrange_input(X.T, 10)

In [11]:
data = input.T

In [12]:
data.size()

torch.Size([1000, 10])

In [20]:
b = torch.zeros(len(data) - 10, 10, data.shape[1],
                    dtype=torch.float64, device=data.device)

In [21]:
b.size()

torch.Size([990, 10, 10])

In [17]:
def make_dataset(model_pred, True_target, context):
    
    # input 은 해결
    Y = True_target.T
    Rx = True_target - model_pred

    assert context >= 1 and isinstance(context, int)
    input = torch.zeros(Y.size(-1), Y.size(0) - context, context, Y.size(-1) - 1,
                        dtype=torch.float32, device = device)
    target = torch.zeros(Y.size(0) - context, context, Y.size(-1),
                         dtype=torch.float32, device = device)
    R_input = torch.zeros_like(target)
    
    ##################################################################################
    
    for i in range(Y.size(-1)):
        
        if i == 0:
            Rx_feature = Rx[1:].T
        elif i == Y.size(0) - 1:
            Rx_feature = Rx[:-1].T
        else:
            Rx_feature = torch.cat((Rx[:i], Rx[i+1:])).T

        for j in range(context):
            start = j
            end = X.size(-1) - context + j
            input[i, :, j, :] = Rx_feature[start:end] 
            target[:, j, :] = Y[start+1:end+1]
            R_input[:, j, :] = model_pred.T[start:end]

    return input.detach(), target.detach(), R_input.detach()

In [18]:
model_pred.size()

torch.Size([10, 1000])

In [19]:
input, target, R_input = make_dataset(model_pred, X, 10)

In [20]:
print(input.size())
print(target.size())
print(R_input.size())

torch.Size([10, 990, 10, 9])
torch.Size([990, 10, 10])
torch.Size([990, 10, 10])


In [22]:
Y = input.T
X = input - R_X
X_feature = X[1:].T

In [21]:
class LSTM(nn.Module):
    def __init__(self, num_series, hidden):
        '''
        LSTM model with output layer to generate predictions.
        Args:
          num_series: number of input time series.
          hidden: number of hidden units.
        '''
        super(LSTM, self).__init__()
        self.p = num_series
        self.hidden = hidden

        # Set up network.
        self.lstm = nn.LSTM(num_series, hidden, batch_first=True)
        self.lstm.flatten_parameters()
        self.linear = nn.Conv1d(hidden, 1, 1)

    def init_hidden(self, batch):
        '''Initialize hidden states for LSTM cell.'''
        device = self.lstm.weight_ih_l0.device
        return (torch.zeros(1, batch, self.hidden, device=device),
                torch.zeros(1, batch, self.hidden, device=device))

    def forward(self, X, hidden=None):
        # Set up hidden state.
        if hidden is None:
            hidden = self.init_hidden(X.shape[0])

        # Apply LSTM.
        X, hidden = self.lstm(X, hidden)

        # Calculate predictions using output layer.
        X = X.transpose(2, 1)
        X = self.linear(X)
        return X.transpose(2, 1), hidden

In [26]:
lstm = LSTM(9,10).to(device)

In [27]:
pred, hid = lstm(input[0])

In [25]:
pred.size()

torch.Size([990, 10, 1])

In [ ]:
input, target, R_input = make_dataset(model_pred, X, 10)

In [28]:
R_input.size()

torch.Size([990, 10, 10])

In [32]:
R_input[:,:,1].size()

torch.Size([990, 10])

In [29]:
pred[:,:,0] + R_input[:,:,0]

tensor([[0.5520, 0.3458, 0.3096,  ..., 1.2245, 0.8038, 1.0267],
        [0.3432, 0.3069, 0.4932,  ..., 0.8042, 1.0267, 0.3991],
        [0.3379, 0.5191, 0.4669,  ..., 1.0270, 0.3993, 0.7455],
        ...,
        [0.0623, 0.9484, 0.8254,  ..., 0.5040, 0.0643, 0.5523],
        [0.9573, 0.8179, 0.9048,  ..., 0.0642, 0.5523, 1.0576],
        [0.8075, 0.9410, 0.0548,  ..., 0.5532, 1.0578, 0.4516]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [39]:
class RBFGC(nn.Module):
    def __init__(self, infeature, hidden, device):
        super(RBFGC, self).__init__()

        self.infeature = infeature
        self.hidden = hidden
        self.device = device
        self.networks = nn.ModuleList([LSTM(infeature-1, hidden) for _ in range(self.infeature)])
    
    def forward(self, X):
        
        if hidden is None:
            hidden = [None for _ in range(self.infeature)]
        pred = [self.networks[i](X[i], hidden[i]) for i in range(self.infeature)]
        pred, hidden = zip(*pred)
        pred = torch.cat(pred, dim=2)
        return pred, hidden

In [48]:
lstm_ori = LSTM(10,10).to(device)

In [71]:
X_np, GC = simulate_lorenz_96(p=10, F=10, T=1000)
X2 = torch.tensor(X_np[np.newaxis], dtype=torch.float32, device=device)

In [72]:
X2.size()

torch.Size([1, 1000, 10])

In [74]:
input_ori, target_ori = zip(*[arrange_input(x, 10) for x in X2])

In [77]:
input_ori[0].size()

torch.Size([990, 10, 10])

In [82]:
target_ori[0].size()

torch.Size([990, 10, 10])

In [78]:
input_ori = torch.cat(input_ori, dim = 0)

In [79]:
input_ori.size()

torch.Size([990, 10, 10])

In [53]:
X.size()

torch.Size([10, 1000])

In [58]:
lstm_ori(input_ori)[0].size()

torch.Size([990, 10, 1])

In [59]:
target_ori.size()

torch.Size([990, 10, 10])

In [33]:
def make_dataset(RBF_model_pred, True_target, context):
    
    # input 은 해결
    Y = True_target.T
    Rx = True_target - RBF_model_pred

    assert context >= 1 and isinstance(context, int)
    input = torch.zeros(Y.size(-1), Y.size(0) - context, context, Y.size(-1) - 1,
                        dtype=torch.float32, device = device)
    target = torch.zeros(Y.size(0) - context, context, Y.size(-1),
                         dtype=torch.float32, device = device)
    R_input = torch.zeros_like(target)
    
    ##################################################################################
    
    for i in range(Y.size(-1)):
        
        if i == 0:
            Rx_feature = Rx[1:].T
        elif i == Y.size(0) - 1:
            Rx_feature = Rx[:-1].T
        else:
            Rx_feature = torch.cat((Rx[:i], Rx[i+1:])).T

        for j in range(context):
            start = j
            end = X.size(-1) - context + j
            input[i, :, j, :] = Rx_feature[start:end] 
            target[:, j, :] = Y[start+1:end+1]
            R_input[:, j, :] = RBF_model_pred[start:end]

    return input.detach(), target.detach(), R_input.detach()

def train_model_adam(clstm, X, RBF_model_pred, context, lr, max_iter, lam=0, lam_ridge=0,
                     lookback=5, check_every=50, verbose=1):
    '''Train model with Adam.'''
    p = X.shape[-1]
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(clstm.parameters(), lr=lr)
    train_loss_list = []

    # Set up data.
    X, Y, R_X = make_dataset(RBF_model_pred, X, context)
    # X = torch.cat(X, dim=0)
    # Y = torch.cat(Y, dim=0)

    # For early stopping.
    best_it = None
    best_loss = np.inf
    best_model = None

    for it in range(max_iter):
        # Calculate loss.
        pred = [clstm.networks[i](X)[0] for i in range(p)]
        loss = sum([loss_fn(pred[i][:, :, 0] + R_X[:,:,i], Y[:, :, i]) for i in range(p)])

        # Add penalty term.
        if lam > 0:
            loss = loss + sum([regularize(net, lam) for net in clstm.networks])

        if lam_ridge > 0:
            loss = loss + sum([ridge_regularize(net, lam_ridge)
                               for net in clstm.networks])

        # Take gradient step.
        loss.backward()
        optimizer.step()
        clstm.zero_grad()

        # Check progress.
        if (it + 1) % check_every == 0:
            mean_loss = loss / p
            train_loss_list.append(mean_loss.detach())

            if verbose > 0:
                print(('-' * 10 + 'Iter = %d' + '-' * 10) % (it + 1))
                print('Loss = %f' % mean_loss)

            # Check for early stopping.
            if mean_loss < best_loss:
                best_loss = mean_loss
                best_it = it
                best_model = deepcopy(clstm)
            elif (it - best_it) == lookback * check_every:
                if verbose:
                    print('Stopping early')
                break

    # Restore best model.
    restore_parameters(clstm, best_model)

    return train_loss_list
